# 使用tf.summary.scalar

In [3]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# 加上tf.reset_default_graph() 就不会报错了

In [22]:
tf.reset_default_graph()
#1.下载mnist数据集
mnist = input_data.read_data_sets('MNIST_data',one_hot=True) #让label转换成one-hot格式

#2对数据进行操作
#2.1分成多个batch
batch_size = 100
n_batch = mnist.train.num_examples // batch_size  #//整除

def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean)#平均值
    with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
    tf.summary.scalar('stddev',stddev)#标准差
    tf.summary.scalar('max',tf.reduce_max(var))
    tf.summary.scalar('min',tf.reduce_min(var))
    tf.summary.histogram('histogram',var)

#2.2为data 和 label 创建plcaeholder   #！None 具体多少数量之后指定
#命名空间
with tf.name_scope(name='input'):
    data = tf.placeholder(tf.float32,[None,784],name='data')
    label = tf.placeholder(tf.float32,[None,10],name='label')
keep_prob = tf.placeholder(tf.float32)
learn = tf.Variable(0.001,dtype=tf.float32)
#3.创建神经网络
#3.1权重参数 和 偏置项

#输出层
with tf.name_scope('layer'):
    with tf.name_scope('weight'):
        W = tf.Variable(tf.truncated_normal([784,10],stddev=0.1),name='W')
        variable_summaries(W)
    with tf.name_scope('biases'):
        b = tf.Variable(tf.zeros([10])+0.1,name='b')
        variable_summaries(b)
    #3.2softmax 为分类求出概率
    with tf.name_scope('mul_add'):
        mul_add = tf.matmul(data,W)+b
    with tf.name_scope('softmax'):
        prediction = tf.nn.softmax(mul_add) 

#3.3使用二次代价函数计算loss值
# loss = tf.reduce_mean(tf.square(prediction-label))
#使用交叉熵 计算loss\
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=label,logits=prediction))
    tf.summary.scalar('loss',loss)

#3.4使用梯度下降法
with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(learn).minimize(loss)

#4.求出准确率
#4.1 得出一个判断正确与否的 布尔类型list
#! arg_max（） 返回一维张量中最大的索引值 1表示按行比较   这里 前一个就是取出one-hot的那个1所在的位置 后一个就是取出概率值最大的位置
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(label,1),tf.argmax(prediction,1))
#4.2 计算正确率   将布尔型列表转换成 0 1 求出平均值就是正确率
    with tf.name_scope('accuracy'):
        accuracy = (tf.reduce_mean(tf.cast(correct_prediction,tf.float32)))
        tf.summary.scalar('accuracy',accuracy)


Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


# 合并所有的指标

In [23]:
merge = tf.summary.merge_all()

In [24]:
#5. session
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #将图保存下来
    writer = tf.summary.FileWriter('logs/',sess.graph)
    for epoch in range(31):
        sess.run(tf.assign(learn,learn*(0.95**epoch))) 
        for batch in range(n_batch):
            #每次向下取出一个batch的数据
            train_data,train_label = mnist.train.next_batch(batch_size) #？为什么这里mnist 有 train方法...
            summary,_ = sess.run([merge,train_step],feed_dict={data:train_data,label:train_label,keep_prob:1.0})
            #传入测试集的数据和标签
        writer.add_summary(summary,epoch)
        test_acc = sess.run(accuracy,feed_dict={data:mnist.test.images,label:mnist.test.labels,keep_prob:1.0})
        train_acc = sess.run(accuracy,feed_dict={data:mnist.train.images,label:mnist.train.labels,keep_prob:1.0})
        print(epoch,"train:",train_acc,"test:",test_acc)

0 train: 0.88714546 test: 0.8964
1 train: 0.9043273 test: 0.9107
2 train: 0.91185457 test: 0.9154
3 train: 0.9157636 test: 0.9179
4 train: 0.9186 test: 0.92
5 train: 0.9193636 test: 0.9225
6 train: 0.92056364 test: 0.9225
7 train: 0.92178184 test: 0.9221
8 train: 0.9219273 test: 0.923
9 train: 0.92225456 test: 0.9229
10 train: 0.92227274 test: 0.9228
11 train: 0.9223273 test: 0.9229
12 train: 0.9224 test: 0.9231
13 train: 0.92243636 test: 0.9231
14 train: 0.9224727 test: 0.9232
15 train: 0.9224909 test: 0.9231
16 train: 0.9224909 test: 0.9231
17 train: 0.9224909 test: 0.9231
18 train: 0.9224909 test: 0.9231
19 train: 0.9224909 test: 0.9231
20 train: 0.9224909 test: 0.9231
21 train: 0.9224909 test: 0.9231
22 train: 0.9224909 test: 0.9231
23 train: 0.9224909 test: 0.9231
24 train: 0.9224909 test: 0.9231
25 train: 0.9224909 test: 0.9231
26 train: 0.9224909 test: 0.9231
27 train: 0.9224909 test: 0.9231
28 train: 0.9224909 test: 0.9231
29 train: 0.9224909 test: 0.9231
30 train: 0.9224909 te